In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from model.vae import VAE

In [12]:
FILE_SYNTH_METADATA = ".\\Datasets\\basses_metadata.csv"
FILE_MODEL_OUTPUT = ".\\Models\\basses_weights"

In [3]:
data = pd.read_csv(FILE_SYNTH_METADATA)
display(data.sample(10))
data.shape

,amp_attack,amp_decay,amp_release,amp_sustain,arp_frequency,arp_gate,arp_octaves,arp_on,arp_pattern,arp_sync,...,Modulation13Amount,Modulation14Source,Modulation14Destination,Modulation14Amount,Modulation15Source,Modulation15Destination,Modulation15Amount,Modulation16Source,Modulation16Destination,Modulation16Amount
42,0.109545,1.376435,1.211073,0.290000,2,1.000000,2,0,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,0.000000,1.500000,0.000000,1.000000,2,0.860000,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,0.117188,1.647059,0.191016,0.000000,2,0.500000,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,0.433620,1.387493,0.466417,0.189269,2,0.431606,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,0.351562,1.294118,0.065625,0.000000,2,0.236000,4,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,0.176471,1.470588,0.000000,0.000000,2,0.200000,2,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,0.176471,0.000000,0.000000,1.000000,2,0.200000,2,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,0.000000,1.941176,0.117188,0.000000,2,0.000000,4,0,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,0.176471,0.000000,0.000000,1.000000,2,0.200000,2,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,0.000000,1.500000,0.000000,1.000000,2,0.500000,1,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(63, 190)

In [4]:
np_data = data.to_numpy()
np_data = np.nan_to_num(np_data, nan=-1.0)
display(np_data.shape)

(63, 190)

In [5]:
model = VAE(latent_dim=6, input_size=np_data.shape[1], version=1)
mean, logvar = model.encode(np_data[0].reshape(1, np_data.shape[1]))
display(mean, logvar)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.12090427, 0.5041382 , 0.47150242]], dtype=float32)>

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.12445959, 0.18972541, 0.47424716]], dtype=float32)>

In [6]:
z = model.reparameterize(mean, logvar)
display(z)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.00090672,  0.38013598,  0.7636409 ]], dtype=float32)>

In [7]:
data_copy = np_data.copy()
np.random.shuffle(data_copy)

split_index = round(len(data_copy)*0.8)
x_train = data_copy[:split_index].copy()
x_test = data_copy[split_index:].copy()

x_train.shape, x_test.shape

((50, 190), (13, 190))

In [8]:
model = VAE(latent_dim=6, input_size=np_data.shape[1], version=1)

In [9]:
optimizer = tf.keras.optimizers.Adam(1e-4)

losses = []
for epoch in range(1, 1000):
  VAE.train_step(model, np_data, optimizer)
  loss = tf.keras.metrics.Mean()
  for i in range(x_test.shape[0]):
    loss(model.compute_loss(x_test))
  losses.append(loss.result())
  if epoch % 50 == 0:
    display("Epoch: {}, Loss: {}".format(epoch, loss.result()))

'Epoch: 50, Loss: 79.94879913330078'

'Epoch: 100, Loss: 54.53045654296875'

'Epoch: 150, Loss: 50.89286804199219'

'Epoch: 200, Loss: 49.20827865600586'

'Epoch: 250, Loss: 48.907745361328125'

...

'Epoch: 750, Loss: 45.78319549560547'

'Epoch: 800, Loss: 45.72562789916992'

'Epoch: 850, Loss: 45.183624267578125'

'Epoch: 900, Loss: 45.28809356689453'

'Epoch: 950, Loss: 45.557762145996094'

In [10]:
z = model.reparameterize(tf.constant([[0.5, 0.5, 0.5]]), tf.constant([[0.5,0.5,0.5]]))
generated_synth = tf.round(model.decode(z))
pd.DataFrame(generated_synth, columns=data.columns)

,amp_attack,amp_decay,amp_release,amp_sustain,arp_frequency,arp_gate,arp_octaves,arp_on,arp_pattern,arp_sync,...,Modulation13Amount,Modulation14Source,Modulation14Destination,Modulation14Amount,Modulation15Source,Modulation15Destination,Modulation15Amount,Modulation16Source,Modulation16Destination,Modulation16Amount
0,-0.0,1.0,0.0,0.0,2.0,0.0,2.0,-0.0,0.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [13]:
model.save_weights(FILE_MODEL_OUTPUT)